In [12]:
import requests
from bs4 import BeautifulSoup

def get_rubber_prices():
    url = "https://www.thitruonghanghoa.com/gia-hang-hoa/gia-cao-su-the-gioi"
    response = requests.get(url)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Tìm thẻ <span class='current-price'>
        price_span = soup.find('span', class_='current-price')
        
        if price_span:
            price = price_span.text.strip()
            return price
        else:
            print("Không tìm thấy giá.")
            return None
    else:
        print(f"Lỗi khi lấy dữ liệu từ trang: {response.status_code}")
        return None

# Gọi hàm lấy giá cao su
rubber_price = get_rubber_prices()

# In kết quả
if rubber_price:
    print(f"Giá cao su hiện tại: {rubber_price}")


Giá cao su hiện tại: 203.50 JPY / kg


In [ ]:
import socket

# Thiết lập máy chủ
HOST = '0.0.0.0'  # Lắng nghe trên mọi địa chỉ
PORT = 12345       # Cổng sử dụng

server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.bind((HOST, PORT))
server_socket.listen(1)

def parse_message(message):
    """
    Phân tích chuỗi nhận được từ ESP32 và chuyển thành dictionary.

    Args:
        message (str): Chuỗi dữ liệu nhận được, ví dụ: "ID: 123, hàm lượng: 45.67"

    Returns:
        dict: Dictionary chứa 'id' và 'ham_luong'.
              Ví dụ: {"id": 123, "ham_luong": 45.67}
    """
    try:
        print(message)
        # Tách các phần của chuỗi dựa trên dấu phẩy
        parts = message.split(",")
        print(parts)
        # Lấy giá trị ID
        id_part = parts[0].split(":")[-1].strip()
        id_value = int(id_part)  # Chuyển đổi thành số nguyên
        print(id_part)
        print(type(id_value))
        # Lấy giá trị hàm lượng
        hamluong_part = parts[1].split(":")[-1].strip()
        hamluong_value = float(hamluong_part)  # Chuyển đổi thành số thực

        # Trả về kết quả dưới dạng dictionary
        return {"id": id_value, "ham_luong": hamluong_value}

    except (IndexError, ValueError) as e:
        # Xử lý lỗi nếu chuỗi không đúng định dạng
        print(f"Lỗi phân tích chuỗi: {e}")
        return None
    
print(f"Server đang lắng nghe trên {HOST}:{PORT}")

while True:
    client_socket, client_address = server_socket.accept()
    print(f"Kết nối từ {client_address}")

    while True:
        data = client_socket.recv(1024)  # Nhận dữ liệu
        if not data:
            break
        print(f"Nhận được: {parse_message(data.decode())}")
        client_socket.sendall(data)  # Gửi lại dữ liệu (echo)

    client_socket.close()
    print(f"Ngắt kết nối từ {client_address}")

Server đang lắng nghe trên 0.0.0.0:12345


In [4]:
import socket
from zeroconf import Zeroconf, ServiceInfo

def start_mdns_service():
    """Khởi tạo mDNS service"""
    zeroconf = Zeroconf()
    service_type = "_http._tcp.local."
    service_name = "myserver._http._tcp.local."
    service_info = ServiceInfo(
        service_type,
        service_name,
        addresses=[socket.inet_aton("192.168.1.188")],  # Địa chỉ IP của máy chủ
        port=12345,
        properties={},
    )
    zeroconf.register_service(service_info)
    print("mDNS service started: myserver.local")
    return zeroconf

def parse_message(message):
    """Phân tích chuỗi nhận được từ ESP32"""
    try:
        parts = message.strip().split(",")
        id_part = parts[0].split(":")[1].strip()
        ham_luong_part = parts[1].split(":")[1].strip()

        data = {
            "id": int(id_part),
            "ham_luong": float(ham_luong_part),
        }
        return data
    except (IndexError, ValueError) as e:
        print(f"Lỗi khi phân tích chuỗi: {e}")
        return None

def start_server(host, port):
    """Khởi động máy chủ TCP"""
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_socket.bind((host, port))
    server_socket.listen(5)
    print(f"Server đang chạy trên {host}:{port}")

    while True:
        client_socket, client_address = server_socket.accept()
        print(f"Kết nối từ {client_address}")
        with client_socket:
            while True:
                data = client_socket.recv(1024)
                if not data:
                    break
                message = data.decode("utf-8")
                print(f"Nhận được: {message}")

                parsed_data = parse_message(message)
                if parsed_data:
                    print(f"Phân tích thành: {parsed_data}")

                # Gửi phản hồi lại ESP32
                response = "Dữ liệu đã nhận!"
                client_socket.sendall(response.encode("utf-8"))

if __name__ == "__main__":
    try:
        zeroconf = start_mdns_service()
        start_server("0.0.0.0", 12345)  # Lắng nghe trên tất cả các địa chỉ
    except KeyboardInterrupt:
        print("\nĐang dừng dịch vụ...")
    finally:
        if zeroconf:
            zeroconf.close()


NameError: name 'zeroconf' is not defined

In [23]:
from flask import Flask, render_template, request, redirect, url_for, flash, session
import pymysql.cursors
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import mysql.connector
import logging
app = Flask(__name__)
db_config = {
    'host': 'localhost',
    'user': 'root',
    'password': 'quocdanh',  # Thay bằng mật khẩu MySQL của bạn
    'database': 'project',  # Thay bằng tên database của bạn
    'charset': 'utf8mb4',
    'cursorclass': pymysql.cursors.DictCursor
}
def generate_user_id():
    """
    Sinh user_id mới dựa trên giá trị lớn nhất trong database.
    """
    try:
        with pymysql.connect(**db_config) as conn:
            cursor = conn.cursor()
            # Lấy giá trị user_id lớn nhất trong bảng
            cursor.execute("SELECT MAX(user_id) FROM user")
            result = list(cursor.fetchone().values())[0]
            
            print(result)
            if result is None:  # Nếu bảng rỗng, bắt đầu từ 1
                next_id = 1
            else:
                next_id = result + 1  # Tăng thêm 1 so với giá trị lớn nhất

            if next_id > 999:  # Giới hạn tối đa là 3 chữ số
                raise ValueError("Đã hết ID khả dụng (vượt quá 3 chữ số).")

            return next_id
    except pymysql.MySQLError as err:
        logging.error(f"Lỗi MySQL: {err}")
        return None
print(generate_user_id())

2
3


In [1]:
def format_vietnamese_currency(value):
    """
    Định dạng số thành tiền tệ Việt Nam:
    - Thêm dấu chấm để phân cách hàng nghìn.
    - Đổi dấu thập phân từ '.' thành ','.
    """
    if isinstance(value, (int, float)):
        # Định dạng giá trị thành chuỗi với phân cách hàng nghìn
        formatted_value = f"{value:,.2f}".replace(",", "X").replace(".", ",").replace("X", ".")
        return formatted_value
    return value
print(format_vietnamese_currency(132123213.00))

132.123.213,00


In [4]:
from flask import Flask
from flask_mail import Mail, Message

app = Flask(__name__)

# Cấu hình email
app.config['MAIL_SERVER'] = 'smtp.gmail.com'
app.config['MAIL_PORT'] = 465
app.config['MAIL_USE_TLS'] = True
app.config['MAIL_USERNAME'] = 'ruppercompany@gmail.com'
app.config['MAIL_PASSWORD'] = 'qsuyeefmfehgfloa'  # App Password cho Gmail.

mail = Mail(app)

@app.route('/send_code')
def send_code():
    msg = Message('Mã xác thực của bạn',
                  sender='ruppercompany@gmail.com',
                  recipients=['iwastheonlygl01@gmail.com'])
    msg.body = "Mã xác thực của bạn là: 123456"
    mail.send(msg)
    return "Email đã được gửi!"
send_code()

SMTPServerDisconnected: Connection unexpectedly closed

In [5]:
from flask import Flask
from flask_mail import Mail, Message

app = Flask(__name__)

# Cấu hình email
app.config['MAIL_SERVER'] = 'smtp.gmail.com'
app.config['MAIL_PORT'] = 587
app.config['MAIL_USE_TLS'] = True
app.config['MAIL_USERNAME'] = 'ruppercompany@gmail.com'
app.config['MAIL_PASSWORD'] = 'shen xgga vffn mvrh'  # App Password.

mail = Mail(app)

@app.route('/send_code')
def send_code():
    msg = Message('Mã xác thực của bạn',
                  sender='ruppercompany@gmail.com',
                  recipients=['iwastheonlygl01@gmail.com'])
    msg.body = "Mã xác thực của bạn là: 123456"
    mail.send(msg)
    return "Email đã được gửi!"

# Gọi send_code trong ứng dụng Flask
if __name__ == "__main__":
    with app.app_context():
        send_code()


In [18]:
from flask import Flask, render_template, request, redirect, url_for, flash, session
import pymysql.cursors
import requests
from bs4 import BeautifulSoup
from datetime import datetime

app = Flask(__name__)
app.secret_key = 'your_secret_key'  # Đổi thành khóa bí mật của bạn

# Cấu hình kết nối MySQL
db_config = {
    'host': 'localhost',
    'user': 'root',
    'password': 'quocdanh',  # Thay bằng mật khẩu MySQL của bạn
    'database': 'project',  # Thay bằng tên database của bạn
    'charset': 'utf8mb4',
    'cursorclass': pymysql.cursors.DictCursor
}

# Hàm lấy vai trò của người dùng từ database
def get_user_role(user_id):
    try:
        conn = pymysql.connect(**db_config)
        with conn.cursor() as cursor:
            query = "SELECT role FROM login WHERE user_id = %s"
            cursor.execute(query, (user_id,))
            result = cursor.fetchone()
        if result:
            return result['role']
        return None
    except pymysql.MySQLError as e:
        print(f"Lỗi MySQL: {e}")
        return None
    finally:
        if conn:
            conn.close()

def get_rubber_prices():
    url = "https://www.thitruonghanghoa.com/gia-hang-hoa/gia-cao-su-the-gioi"
    response = requests.get(url)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Tìm thẻ <span class='current-price'>
        price_span = soup.find('span', class_='current-price')
        
        if price_span:
            price = price_span.text.strip()
            return price
        else:
            print("Không tìm thấy giá.")
            return None
    else:
        print(f"Lỗi khi lấy dữ liệu từ trang: {response.status_code}")
        return None

def update_price_to_db(price):
    # Kết nối tới cơ sở dữ liệu MySQL
    conn = pymysql.connect(**db_config)
    cursor = conn.cursor()
    
    # Lấy ngày hiện tại
    today = datetime.today().date()

    # Đảm bảo giá được định dạng đúng
    price_value = float(price.split()[0])  # Lấy phần số của giá, ví dụ: "203.50" từ "203.50 JPY / kg"

    # Kiểm tra xem giá của ngày hôm nay đã có trong bảng chưa
    cursor.execute("SELECT COUNT(*) FROM price WHERE day = %s", (today,))
    result = cursor.fetchone()

    if result[0] > 0:
        # Nếu đã có dữ liệu, cập nhật giá
        cursor.execute("UPDATE price SET price = %s WHERE day = %s", (price_value, today))
        print(f"Cập nhật giá cao su cho ngày {today}: {price_value}")
    else:
        # Nếu chưa có dữ liệu, chèn giá mới
        cursor.execute("INSERT INTO price (day, price) VALUES (%s, %s)", (today, price_value))
        print(f"Thêm giá cao su cho ngày {today}: {price_value}")
    conn.commit()
    cursor.close()
    conn.close()

# Trang đăng nhập
@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        username = request.form['username']
        password = request.form['password']
        
        try:
            conn = pymysql.connect(**db_config)
            with conn.cursor() as cursor:
                query = "SELECT * FROM login WHERE username = %s AND password = %s"
                cursor.execute(query, (username, password))
                user = cursor.fetchone()

            if user:
                session['user_id'] = user['user_id']

                # Điều hướng theo vai trò người dùng
                user_role = get_user_role(user['user_id'])
                if user_role == 'manager':
                    flash('Đăng nhập thành công, chào mừng Quản lý!', 'success')
                    return redirect(url_for('manager_dashboard'))
                elif user_role == 'customer':
                    flash('Đăng nhập thành công, chào mừng Khách hàng!', 'success')
                    return redirect(url_for('customer_dashboard'))
                else:
                    flash('Role không hợp lệ!', 'danger')
                    return redirect(url_for('login'))
            else:
                flash('Sai tên đăng nhập hoặc mật khẩu!', 'danger')
        except pymysql.MySQLError as e:
            flash(f"Lỗi cơ sở dữ liệu: {e}", 'danger')
        finally:
            if conn:
                conn.close()

    return render_template('index.html')

@app.route('/forgot_password', methods=['GET', 'POST'])
def forgot_password():
    if request.method == 'POST':
        email = request.form['email']
        flash('Nếu email tồn tại, bạn sẽ nhận được hướng dẫn đặt lại mật khẩu!', 'info')
        return redirect(url_for('login'))
    return render_template('forgot_password.html')
# Dashboard cho quản lý
@app.route('/manager_dashboard')
def manager_dashboard():
    if 'user_id' not in session:
        return redirect(url_for('login'))

    # Cập nhật giá cao su khi vào dashboard
    rubber_price = get_rubber_prices()
    if rubber_price:
        update_price_to_db(rubber_price)
    
    return render_template('manager_dashboard.html')

# Dashboard cho khách hàng
@app.route('/customer_dashboard')
def customer_dashboard():
    if 'user_id' not in session:
        return redirect(url_for('login'))
    
    return render_template('customer_dashboard.html')

# Đăng xuất
@app.route('/logout')
def logout():
    session.pop('user_id', None)
    flash('Đăng xuất thành công!', 'success')
    return redirect(url_for('login'))

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1

c:\Users\21139\miniconda3\envs\project_2\lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
import requests
from bs4 import BeautifulSoup
import mysql.connector
from datetime import datetime

db_config = {
    'host': 'localhost',
    'user': 'root',
    'password': 'quocdanh',  # Thay bằng mật khẩu MySQL của bạn
    'database': 'project',  # Thay bằng tên database của bạn
    'charset': 'utf8mb4',
    'cursorclass': pymysql.cursors.DictCursor
}

# Hàm lấy giá cao su từ trang web
def get_rubber_prices():
    url = "https://www.thitruonghanghoa.com/gia-hang-hoa/gia-cao-su-the-gioi"
    response = requests.get(url)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Tìm thẻ <span class='current-price'>
        price_span = soup.find('span', class_='current-price')
        
        if price_span:
            price = price_span.text.strip()
            return price
        else:
            print("Không tìm thấy giá.")
            return None
    else:
        print(f"Lỗi khi lấy dữ liệu từ trang: {response.status_code}")
        return None

# Hàm kết nối cơ sở dữ liệu và cập nhật giá vào bảng
def update_price_to_db(price):
    # Kết nối tới cơ sở dữ liệu MySQL
    conn = mysql.connector.connect(
        host="localhost",  # Địa chỉ máy chủ MySQL
        user="root",  # Tên người dùng MySQL
        password="quocdanh",  # Mật khẩu MySQL
        database="project"  # Tên cơ sở dữ liệu
    )
    
    cursor = conn.cursor()
    
    # Lấy ngày hiện tại
    today = datetime.today().date()

    # Đảm bảo giá được định dạng đúng
    price_value = float(price.split()[0])  # Lấy phần số của giá, ví dụ: "203.50" từ "203.50 JPY / kg"

    # Kiểm tra xem giá của ngày hôm nay đã có trong bảng chưa
    cursor.execute("SELECT COUNT(*) FROM price WHERE day = %s", (today,))
    result = cursor.fetchone()
    print(result)
    if result[0] > 0:
        # Nếu đã có dữ liệu, cập nhật giá
        cursor.execute("UPDATE price SET price = %s WHERE day = %s", (price_value, today))
        print(f"Cập nhật giá cao su cho ngày {today}: {price_value}")
    else:
        # Nếu chưa có dữ liệu, chèn giá mới
        cursor.execute("INSERT INTO price (day, price) VALUES (%s, %s)", (today, price_value))
        print(f"Thêm giá cao su cho ngày {today}: {price_value}")
    
    # Commit thay đổi và đóng kết nối
    conn.commit()
    cursor.close()
    conn.close()

# Gọi hàm lấy giá cao su
rubber_price = get_rubber_prices()

# Nếu lấy được giá, cập nhật vào cơ sở dữ liệu
if rubber_price:
    update_price_to_db(rubber_price)


{'COUNT(*)': 1}


KeyError: 0

In [6]:
import requests

# URL của trang web cần crawl
url = "https://www.thitruonghanghoa.com/gia-hang-hoa/gia-cao-su-the-gioi"

# Gửi yêu cầu GET để lấy nội dung trang web
response = requests.get(url)

# Kiểm tra nếu yêu cầu thành công
if response.status_code == 200:
    # Lưu nội dung HTML vào file
    with open("rubber_prices.html", "w", encoding="utf-8") as file:
        file.write(response.text)
    print("Đã lưu HTML vào file rubber_prices.html")
else:
    print(f"Lỗi khi tải trang web: {response.status_code}")


Đã lưu HTML vào file rubber_prices.html


In [ ]:
from bs4 import BeautifulSoup

# Giả sử bạn có HTML như sau:
html = "<span class='current-price'>203.50 JPY / kg</span>"

# Parse HTML
soup = BeautifulSoup(html, 'html.parser')

# Tìm thẻ có class 'current-price' và lấy nội dung
price = soup.find('span', class_='current-price').text

# In kết quả
print(price)  # Output: 203.50 JPY / kg
